In [6]:
import pandas as pd
import numpy as np
import time
from copy import deepcopy

In [3]:
df = pd.read_csv('items.csv',header = None)
df.rename(columns = {0: 'weight'}, inplace = True)
df['item_no'] = [i for i in range(len(df))]
df = df.iloc[:, [1,0]]

In [4]:
weight_list = [i for i in df['weight']]
len(weight_list)

249

In [5]:
item_dict = dict({})
for i,value in enumerate(weight_list):
    item_dict[i] = value

In [7]:
def shuffle_item_dict(data):
    """
    Shuffle the items in a dictionary.

    Parameters:
        data (dict): A dictionary to be shuffled.

    Returns:
        dict: A new dictionary with items shuffled.
    """
    
    keys = list(data.keys())
    
    np.random.shuffle(keys)
    
    data = {key : data[key] for key in keys}
    
    return data

In [8]:
def calculate_bin_weight(data, bins_collection, bin_no):
    """
    Calculate the total weight of items in a specified bin.

    Parameters:
    - data (dict): Dictionary containing the item no. and weights.
    - bins (dict): A dictionary representing a configuration, where keys are bin no. and values are lists of items in each bin.
    - bin_no (int): The index of the bin for which the total weight needs to be calculated.

    Returns:
    float: The total weight of items in the specified bin, rounded to one decimal place.
    """
    return round(sum([data[key] for key in bins_collection[bin_no]]),1)

In [21]:
def fitness(data, candidate_solution, bin_size):
    """
    Calculate the fitness score for a candidate solution.

    Parameters:
        data (list): List of item weights.
        candidate_solution (list): Candidate solution represented as a list of bins.
        bin_size (int): Size of each bin.

    Returns:
        float: Fitness score for the candidate solution.
    """
    
    score = 0
    # Iterate through each bin in the candidate solution
    for bin_no in candidate_solution:
        # Calculate the fitness contribution for the current bin
        score_i = ((calculate_bin_weight(data, candidate_solution, bin_no) / bin_size)**2) / len(candidate_solution)

        # Sum the fitness
        score += score_i
        
    return round(score,3)

In [16]:
def move(data, individual, bin_size = 100):
    mu_individual = deepcopy(individual)

    bin_no = len(mu_individual)
    for gene in list(mu_individual):

        #Eliminating a random gene
        item_list = sorted(mu_individual.pop(gene), reverse = True)
        
        for item in item_list:
            # Retrieve the item no.
            current_item_no = item
    
            # Retrieve the item weight
            current_weight = data[item]
            
            total_weight = 0
    
            # Iterate through existing bins
            for gene in mu_individual:              
                # Calculate the total weight of the randomly selected bin
                bin_total_weight = calculate_bin_weight(data, mu_individual, gene)
                
                # Add the weight of the current unique item to the weight of the randomly selected bin
                total_weight = round((bin_total_weight + current_weight), 1)
                
                # Check if there is available space within the randomly selected bin
                if total_weight <= bin_size:
                    
                    # Adding the item to the bin
                    mu_individual[gene].append(current_item_no)
                    break
    
            # Opening a new bin if the old bins are full
            if total_weight > bin_size:
                bin_no += len(mu_individual)+1
                mu_individual.update({bin_no: [current_item_no]})
    
        #Removing any empty bins
        mu_individual = {bin: item for bin,item in mu_individual.items() if item}

    # Renaming the bins correctly
    new_keys = list(range(1,len(mu_individual)+1))
    for key,new_key in zip(sorted(mu_individual.keys()), new_keys):
        mu_individual[new_key] = mu_individual.pop(key)

    return mu_individual

In [12]:
def initialise_candidate(data, bin_size):
    """
    Generate an initial solution using the random fitting approach.

    Parameters:
    - data (dict): Dictionary containing the item no. and weights.
    - bin_size (int): Maximum capacity of each bin.

    Returns:
    dict: A dictionary representing the initial configuration, where keys are bin no. and values are lists of items in each bin.
    """
    bin_no = 1
    init_bins = {bin_no: []}  # Opening the first bin

    # Shuffling the item_dict
    data = shuffle_item_dict(data=data)
    
    # Iterate through all the unique items
    for item_no, item_weight in item_dict.items():
        # Retrieving the item no.
        current_item_no = item_no
        
        # Retrie the item weight
        current_weight = item_weight
        
        total_weight = 0

        # Iterate through existing bins
        for _ in init_bins:
            # Randomly select an existing bin
            random_bin = np.random.choice(list(init_bins.keys()))
            
            # Calculating the total weight of the randomly selected bin
            random_bin_total_weight = calculate_bin_weight(data, init_bins, random_bin)
            
            # Add the weight of the current unique item to the weight of the randomly selected bin
            total_weight = round((random_bin_total_weight + current_weight), 1)
            
            # Check if there is available space within the randomly selected bin
            if total_weight <= bin_size:
                
                # Adding the item to the bin
                init_bins[random_bin].append(current_item_no)
                break

            # Opening a new bin if the old bins are full
            if total_weight > bin_size:
                bin_no += 1
                init_bins.update({bin_no: [current_item_no]})
                break
                
    return init_bins

In [15]:
current_solution = initialise_candidate(data=item_dict, bin_size=100)

In [18]:
new_solution = move(data=item_dict, individual=current_solution, bin_size=100)

In [31]:
def hill_climber(data, bin_size, iteration):
    start_time = time.time()
    
    best_solution = deepcopy(initialise_candidate(data=item_dict, bin_size=bin_size))
    current_solution = deepcopy(best_solution)

    for i in range(iteration):
        print('Iteration:', i, '| Current Fitness:', fitness(data, current_solution, bin_size = bin_size),'| Number of bins:', len(current_solution))
        new_solution = move(data=item_dict, individual=current_solution, bin_size=bin_size)

        current_fitness = fitness(data, current_solution, bin_size = bin_size)
        new_fitness = fitness(data, new_solution, bin_size = bin_size)

        if new_fitness > current_fitness:
            current_solution = deepcopy(new_solution)
        else:
            continue
    stop_time = time.time()
    elapsed_time = stop_time - start_time
    elasped_time_mininute = round((elapsed_time/60),2)
    print('---Execution time:', elasped_time_mininute,'minutes---')
    
    return current_solution, elasped_time_mininute

In [28]:
optimal_solution = hill_climber(item_dict, 100, 500)

Iteration: 0 | Current Fitness: 0.503 | Number of bins: 126
Iteration: 1 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 2 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 3 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 4 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 5 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 6 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 7 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 8 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 9 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 10 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 11 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 12 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 13 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 14 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 15 | Current Fitness: 0.82 | Number of bins: 92
Iteration: 16 | Current Fitness: 0.82 | Number of bins: 92
Itera

In [33]:
test_data = {'Run':[],
             'HighestFitness':[],
             'MinimumBin':[],
             'Runtime':[]}
runs = 10
for run in range(1, runs+1):
    print('----------------')
    print('Run no.', run,'\n')
    
    optimal_solution, elapsed_time = hill_climber(item_dict, 100, 500)
    
    test_data['Run'].append(run)
    test_data['HighestFitness'].append(fitness(item_dict, optimal_solution, bin_size=100))
    test_data['MinimumBin'].append(len(optimal_solution))
    test_data['Runtime'].append(elapsed_time)

    print('----Complete----')
print(test_data)

----------------
Run no. 1 

Iteration: 0 | Current Fitness: 0.452 | Number of bins: 133
Iteration: 1 | Current Fitness: 0.805 | Number of bins: 93
Iteration: 2 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 3 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 4 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 5 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 6 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 7 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 8 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 9 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 10 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 11 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 12 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 13 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 14 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 15 | Current Fitness: 0.819 | Number of bins: 92
Iteration: 16 | Curr

In [35]:
collected_data = pd.DataFrame(test_data)
collected_data.to_csv('CollectedDataHC.csv')